In [18]:
import pandas as pd
import numpy as np
decile_data= pd.read_csv('C:\\Users\\zihan\\Desktop\\Files\\Research\\decile.csv',header=None)
date= pd.read_csv('C:\\Users\\zihan\\Desktop\\Files\\Research\\Portfolios.csv', usecols=[0])

In [19]:
path_target = 'C:/Users/zihan/Desktop/Files/Research/decile_data.npz'
print('Saving data to ' + path_target)
np.savez(path_target, decile_data=decile_data, date= date)
print('Finished! ')

Saving data to C:/Users/zihan/Desktop/Files/Research/decile_data.npz
Finished! 


In [22]:
import pandas as pd
import numpy as np
decile_data= np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\decile_data.npz')
decile= decile_data['decile_data']
date= decile_data['date']
date= date[1:]
portfolio= decile
fama_french_data= np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\riskfreerate.npz')
fama_french_data= fama_french_data['fama_french_data']
r_f= fama_french_data[:,4]
fama_french_date= fama_french_data[:,0]
r_f= r_f/100
# date_start= np.asscalar(np.argwhere(fama_french_date==196307))
# date_end= np.asscalar(np.argwhere(fama_french_date==201608))
data_port_start= (np.argwhere(date=='Jun-84'))[0][0]
data_port_end= (np.argwhere(date=='May-14'))[0][0]
# portfolio= portfolio[0:638,:]- (r_f[date_start:date_end+1:,np.newaxis])
date_start= np.asscalar(np.argwhere(fama_french_date==198406))
date_end= np.asscalar(np.argwhere(fama_french_date==201405))
portfolio= portfolio[data_port_start:data_port_end,:]- r_f[date_start:date_end:,np.newaxis]

In [34]:
import scipy.linalg as la
T, M= portfolio.shape
K = 7
window= 120
optimal_return = np.zeros((T-window))
gamma = 10
for t in range(T-window):
    portfolio_t = portfolio[t:(t + window), :]
    covmat_t = portfolio_t.T.dot(portfolio_t) 
    portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t)/window * gamma
    variance_t1, loading_t= la.eigh(covmat_t, eigvals=(M-K, M-1))
    Factor_t= portfolio_t.dot(loading_t)
    mu= np.mean(Factor_t, axis= 0)
    covmat_factor= np.cov(Factor_t.T)
    weight= np.linalg.inv(covmat_factor).dot(mu)
    optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(weight)
mean_optimal= np.mean(optimal_return)
cov_optimal= np.cov(optimal_return)
print (abs(mean_optimal)/np.sqrt(cov_optimal))
window= 120
optimal_return = np.zeros((T-window))
gamma = 100
T, M = portfolio.shape
ones_T= np.ones((window, 1))
# Maybe can just change the T-1 to be T.
for t in range(T-window):
    portfolio_t = portfolio[t:(t + window), :]
    covmat_t= portfolio_t.dot(portfolio_t.T).dot(gamma/window* ones_T.dot(ones_T.T))
    covmat_t= covmat_t+ portfolio_t.dot(portfolio_t.T)
    # variance_t, Factor_t = np.linalg.eig(covmat_t)
    variance_t2, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
    # Factor_t= Factor_t[:,:K]
    loading_t= Factor_t.T.dot(portfolio_t)
    #loading_t: K*N
    mu= np.mean(Factor_t, axis= 0)
    covmat_factor= np.cov(Factor_t.T)
    weight= np.linalg.inv(covmat_factor).dot(mu)
    optimal_return[t] = (portfolio[t + window, :].dot(loading_t.T)).dot(np.linalg.inv(loading_t.dot(loading_t.T))).dot(weight)
mean_optimal= np.mean(optimal_return)
cov_optimal= np.cov(optimal_return)
print (abs(mean_optimal)/np.sqrt(cov_optimal))

0.354333455436
0.126955431924


In [69]:
# Plot the bar plot
def plot_sharpe(K, portfolio):
    import scipy.linalg as la
    T, M= portfolio.shape
    window= 240
    optimal_return = np.zeros((T-window))
    gamma = 10
    for t in range(T-window):
# for t in range(0,1):
#     if t % 20 == 0:
#         print('Have gone through iteration %d' % (t))
        portfolio_t = portfolio[t:(t + window), :]
        covmat_t = portfolio_t.T.dot(portfolio_t) 
        portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
        covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t)/window * gamma
        variance_t, loading_t= la.eigh(covmat_t, eigvals=(M-K, M-1))
    # variance_t, loading_t= np.linalg.eig(covmat_t)
        Factor_t= portfolio_t.dot(loading_t)
        mu= np.mean(Factor_t, axis= 0)
        covmat_factor= np.cov(Factor_t.T)
        if K>=2:
            weight= np.linalg.inv(covmat_factor).dot(mu)
        else:
            weight= 1
        optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(weight)
    mean_optimal= np.mean(optimal_return)
    cov_optimal= np.cov(optimal_return)
    return (abs(mean_optimal)/np.sqrt(cov_optimal))

In [70]:
sharpe_ratio= np.zeros((2,7))
decile= decile_data['decile_data']
portfolio= decile
data_port_start= (np.argwhere(date=='Jun-84'))[0][0]
data_port_end= (np.argwhere(date=='May-14'))[0][0]
# portfolio= portfolio[0:638,:]- (r_f[date_start:date_end+1:,np.newaxis])
date_start= np.asscalar(np.argwhere(fama_french_date==198406))
date_end= np.asscalar(np.argwhere(fama_french_date==201405))
portfolio= portfolio[data_port_start:data_port_end,:]- r_f[date_start:date_end:,np.newaxis]
for i in range(7):
    print (i)
    sharpe_ratio[0,i]= plot_sharpe(i+1, portfolio)
decile= decile_data['decile_data']
portfolio= decile
date_start= np.asscalar(np.argwhere(fama_french_date==196307))
date_end= np.asscalar(np.argwhere(fama_french_date==201608))
portfolio= portfolio[0:638,:]- (r_f[date_start:date_end+1:,np.newaxis])
for i in range(7):
    print (i)
    sharpe_ratio[1,i]= plot_sharpe(i+1, portfolio)

0
1
2
3
4
5
6
0
1
2
3
4
5
6


In [65]:
import pandas as pd
import matplotlib.pyplot as plt
df2 = pd.DataFrame(sharpe_ratio, columns=['1 Factor','2 Factor','3 Factor','4 Factor','5 Factor','6 Factor','7 Factor'], index= ['P1', 'P2'])
df2.plot.bar()
plt.savefig('C://Users//zihan//Desktop//Files//Research//sharpe_comparison.png')
plt.clf()

In [2]:
import scipy.linalg as la
K = 1
window= 120
gamma = 100
T, M = portfolio.shape
pricing_error= np.zeros((T-window,M))
ones_T= np.ones((window, 1))
for t in range(T-window):
# for t in range(0,1):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    covmat_t= portfolio_t.dot(portfolio_t.T).dot(gamma/T* ones_T.dot(ones_T.T))
    covmat_t= covmat_t+ portfolio_t.dot(portfolio_t.T)
    variance_t, Factor_t = np.linalg.eig(covmat_t)
    Factor_t= Factor_t[:,:K]
    loading_t= Factor_t.T.dot(portfolio_t)
    pricing_error[t,:] = (portfolio[t + window, :]- portfolio[t+window, :].dot(loading_t.T).dot(np.linalg.inv(loading_t.dot(loading_t.T))).dot(loading_t))
avg_pricing_error= np.mean(pricing_error*100, axis= 0)
print (np.sqrt(np.sum(avg_pricing_error**2)/M))
# print (np.std(avg_pricing_error))
# print (np.sqrt(np.sum(pricing_error**2)/M))

Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
Have gone through iteration 180
Have gone through iteration 200
Have gone through iteration 220
Have gone through iteration 240
Have gone through iteration 260
Have gone through iteration 280
Have gone through iteration 300
Have gone through iteration 320
Have gone through iteration 340
Have gone through iteration 360
Have gone through iteration 380
Have gone through iteration 400
Have gone through iteration 420
Have gone through iteration 440
Have gone through iteration 460
Have gone through iteration 480
Have gone through iteration 500
0.251463601509


In [39]:
def compute_sharpe(K):
    import scipy.linalg as la
    T, M = portfolio.shape
    window= 120
    optimal_return = np.zeros((T-window))
    gamma = 10    
    ones_T= np.ones((window, 1))
# Maybe can just change the T-1 to be T.
    for t in range(T-window):
# for t in range(0,1):
#         if t % 20 == 0:
#             print('Have gone through iteration %d' % (t))
        portfolio_t = portfolio[t:(t + window), :]
        covmat_t= portfolio_t.dot(portfolio_t.T).dot(gamma/T* ones_T.dot(ones_T.T))
        covmat_t= covmat_t+ portfolio_t.dot(portfolio_t.T)
    # variance_t, Factor_t = np.linalg.eig(covmat_t)
        variance_t, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
        Factor_t= Factor_t[:,:K]
        loading_t= Factor_t.T.dot(portfolio_t)
        mu= np.mean(Factor_t, axis= 0)
        covmat_factor= np.cov(Factor_t.T)
        if K>=2:
            weight= np.linalg.inv(covmat_factor).dot(mu)
        else:
            weight= 1
        optimal_return[t] = (portfolio[t + window, :].dot(loading_t.T)).dot(weight)
        # optimal_return[t]= Factor_t[t,:].dot(weight)
    mean_optimal= np.mean(optimal_return)
    cov_optimal= np.cov(optimal_return)
    return (abs(mean_optimal)/np.sqrt(cov_optimal))
K= 20
optimal_sharpe= np.zeros((K))
for i in range(K):
    optimal_sharpe[i]= compute_sharpe(i+1)
print (optimal_sharpe)

[ 0.01842673  0.14318161  0.14017854  0.14474434  0.14514331  0.14449405
  0.1447339   0.14515375  0.14498881  0.14506717  0.14499331  0.14530811
  0.14515363  0.14501643  0.14487021  0.14507405  0.14477885  0.14486559
  0.14489803  0.14489588]


In [38]:
print(loading_t.shape)

(7, 370)
